In [3]:
#não executar, apenas mandar para o final para adicionar palavras

with open('palavras_termo.txt','w', encoding = 'utf8') as f:
    for palavra in palavras:
        f.write("%s\n" % palavra)
    print('done')


NameError: name 'palavras' is not defined

In [41]:
import pandas as pd
import numpy as np

In [45]:
import unicodedata
import re

def remove_acentos(string):
    normalized = unicodedata.normalize('NFD', string)
    return re.sub(r'[\u0300-\u036f]', '', normalized).casefold()

In [43]:
def conta_vogais(palavra):
    
    contador = 0
    vogais = 'aeiouAEIOU'
    allowed = frozenset('aeiouAEIOU')
    
    for letra in palavra:
        if letra in vogais:
            contador += 1
    vogais_unicas = len(allowed.intersection(palavra))
    
    return vogais_unicas, contador


conta_vogais('perde')    



(1, 2)

In [127]:
palavras = []
file = open('br-utf8.txt', 'r', encoding = 'utf8')

while True:
    line = file.readline()
    palavra = line.strip().replace('\n','')
    if not line:
        break
    if len(palavra) == 5:
        #print(palavra)
        palavras.append(remove_acentos(palavra))
        

file.close()


#nesse caso não tem a palavra lermo, então vamos remover da lista e rodar novamente
#criar uma função para quando isso acontecer e criar uma lista das palavras que o termo não aceita!
#assim podemos ir melhorando o algoritmo

palavras.remove('lermo')
palavras.remove('canaa')
palavras.remove('propo')
palavras.remove('coemo')
palavras.remove('doemo')

#print(len(palavras))
df = pd.DataFrame(palavras)
df.drop_duplicates(inplace = True)
df.rename(columns = {0:'palavra'}, inplace = True)

df['n_vogais'] = df['palavra'].apply(lambda x: conta_vogais(x)[1])
df['vogais_unicas'] = df['palavra'].apply(lambda x: conta_vogais(x)[0])


df['letra_0'] = df.palavra.str[0]
df['letra_1'] = df.palavra.str[1]
df['letra_2'] = df.palavra.str[2]
df['letra_3'] = df.palavra.str[3]
df['letra_4'] = df.palavra.str[4]

df.head()

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
0,aarao,4,2,a,a,r,a,o
1,abner,2,2,a,b,n,e,r
2,acaia,4,2,a,c,a,i,a
3,acker,2,2,a,c,k,e,r
4,aires,3,3,a,i,r,e,s


In [128]:
df.to_csv('br-utf8.csv', index = False)

# Função de sortear palavra
estou usando esse metodo até criar um esquema de pontuação para as palavras

(depois necessário criação de CSV com a pontuação e todo a manipulação de dados, para que no final só fique o filtro!)

In [311]:
import random

def sorteia_palavra(lista_palavra):
    if len(lista_palavra) == 1:
        return lista_palavra[0]
    else:
        return lista_palavra[random.randint(0,(len(lista_palavra)-1))]



In [51]:
melhores_palavras_para_vogais = df[df.vogais_unicas == 4]
#escolher entre essas palavras para começar

# Startando o termoo no navegador
aqui seria onde comecei a entender como fazer a busca pela palavra e tentar criar alguma metrica pra achar letras nas strings


como proceder com palavras que tem letras repetidas?

aqui podemos fazer uma condição para cada posição estar com a letra certa


while pos1 == wrongletter or pos2 == wrongletter or pos3 == wrongletter or pos4 == wrongletter or pos5 == wrongletter

e ir lupando nas tentativas que temos, que aparentemente são 6

In [111]:
df.loc[df.palavra == 'nervo',:]

,palavra,n_vogais,vogais_unicas
3628,nervo,2,2


esse é o pensamento inicial, depois que começar o scraping no site e inserção poderemos saber quando acertarmos a posição da letra. 

Quando isso acontecer será necessário trocar essa função para adicionar só palavras com letra naquela posição!

teoricamente aqui conseguimos resolver mais ou menos o termo, agora é pesquisar novas metricas e testar nos outros dias, melhorando o filtro e a seleção, também seria legal rankear as palavras mais comuns e ver se conseguimos gerar um peso de palavras

- trabalhar em melhorar as funções e como elas vão conversar entre elas

-----------------------------

Palavras excluidas do dicionario baixado da usp:
- lermo
- canaa
- propo
- coemo
- doemo
- jobim
- suamo

-----------------------------

# Organizando todas as funções!

- Carregar o DF
- listar as palavras melhroes rankeadas para começar o jogo
- iniciar o jogo
- escolher uma palavra da lista de mais rankeadas
- mandar a primeira palavra para o jogo
- retornar dicionario de resultado da primeira palavra

## Core Functions - Analitics

In [1]:
import pandas as pd

def carrega_dataframe_csv(caminho):
    return pd.read_csv(caminho)

In [2]:
import random

def sorteia_palavra(lista_palavra):
    if len(lista_palavra) == 1:
        return lista_palavra[0]
    else:
        return lista_palavra[random.randint(0,(len(lista_palavra)-1))]

In [3]:
#recebe lista de letra e posicao correta
#lista_exemplo = ['letra',posicao]

def letra_na_posicao(lista_letra_correta, df):
    
    if len(lista_letra_correta) == 0:
        return df
    else:
        for letra, posicao in lista_letra_correta:
            if posicao == 0:
                mask = df.letra_0 == letra
                df = df.loc[mask,:]
            elif posicao == 1:
                mask = df.letra_1 == letra
                df = df.loc[mask,:]
            elif posicao == 2:
                mask = df.letra_2 == letra
                df = df.loc[mask,:]
            elif posicao == 3:
                mask = df.letra_3 == letra
                df = df.loc[mask,:]
            elif posicao == 4:
                mask = df.letra_4 == letra
                df = df.loc[mask,:]

    return df


In [4]:
#lista_de_letras_com_posicao = [['g',2]] --- exemplo
#provavelmente será necessário criar uma função que descompacte o dicionario que retorna do site em uma lista com listas

def palavras_com_letra_posicao_errada(lista_de_letras_com_posicao, df):   
    lista = []
    for indice1,indice2 in lista_de_letras_com_posicao:
        lista.append(indice1)
        
        condicao = indice2
        if condicao == 0:
            mask = df.letra_1.isin(lista) | df.letra_2.isin(lista) | df.letra_3.isin(lista) | df.letra_4.isin(lista)
            df = df.loc[mask,:]
        elif condicao == 1:
            mask = df.letra_0.isin(lista) | df.letra_2.isin(lista) | df.letra_3.isin(lista) | df.letra_4.isin(lista) 
            df = df.loc[mask,:]
        elif condicao == 2:
            mask = df.letra_0.isin(lista) | df.letra_1.isin(lista) | df.letra_3.isin(lista) | df.letra_4.isin(lista)
            df = df.loc[mask,:]
        elif condicao == 3:
            mask = df.letra_0.isin(lista) | df.letra_1.isin(lista) | df.letra_2.isin(lista) | df.letra_4.isin(lista)
            df = df.loc[mask,:]
        elif condicao == 4:
            mask = df.letra_0.isin(lista) | df.letra_1.isin(lista) | df.letra_2.isin(lista) | df.letra_3.isin(lista)
            df = df.loc[mask,:]

    return df

In [5]:
def letras_nao_aceitas(letras, df):
    lista = []
    
    if len(letras) > 1:
        for i in letras:
            lista.append(i)
        mask = ~df.letra_0.isin(lista) & ~df.letra_1.isin(lista) & ~df.letra_2.isin(lista) & ~df.letra_3.isin(lista) & ~df.letra_4.isin(lista)
    else:
        mask = (df.letra_0 != letras) & (df.letra_1 != letras) & (df.letra_2 != letras) & (df.letra_3 != letras) & (df.letra_4 != letras)

    return df.loc[mask,:]



## Scraping functions

In [6]:
import pandas as pd
import numpy as np
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
#com a mudança pro selenium 4.0, algumas coisas mudaram ao inicializar, por exemplo o uso do Service


from bs4 import BeautifulSoup
from tqdm import tqdm

In [7]:
#inicializand o site
def inicializa_termo():
    service = Service(executable_path = './edg_drv/msedgedriver.exe') #mudança do selenium 4.0
    driver = webdriver.Edge(service=service)

    urlpage = 'https://term.ooo/'
    response = driver.get(urlpage)
    driver.implicitly_wait(5)

    x_path = '/html/body/wc-modal'
    ajuda_sempre_aberta = driver.find_element('xpath',x_path)
    ajuda_sempre_aberta.click()
    driver.implicitly_wait(5)
    return driver
    
    


In [97]:
#envia palavra
from time import sleep

def envia_palavra(palavra_sorteada):
    x_path_board = '/html/body'
    board_inicial = driver.find_element('xpath',x_path_board)
    for letra in palavra_sorteada:
        board_inicial.send_keys(letra)
        #board_inicial.implicitly_wait(3)#testar, se não funcionar voltar para -> driver.implicitly_wait(0.5)
        sleep(1)

    
    board_inicial.send_keys(Keys.ENTER)
    #board_inicial.implicitly_wait(5)
    sleep(3)
    #print('palavra-enviada')#apenas pra finalidades de teste, retirar depois
    
def apaga_palavra():
    x_path_board = '/html/body'
    board_inicial = driver.find_element('xpath',x_path_board)

    for i in range(5):
        board_inicial.send_keys(Keys.BACKSPACE)
        sleep(0.5)
    

In [43]:
#retorna dicionario da tentiva de palavra enviada
# elemento_do_board = '[id="board0"]'

# shadow = driver.find_element(By.CSS_SELECTOR, elemento_do_board).shadow_root

#ainda pensando se vale a pena colocar como variavel global mesmo ou deixar dentro da função

def retorna_dicionario_respostas(variavel): #variavel = numero da tentativa

    elemento_do_board = '[id="board0"]'

    shadow = driver.find_element(By.CSS_SELECTOR, elemento_do_board).shadow_root
    letra = ''
    resposta = ''
    
    
    lista_da_palavra = [] #inicializa a lista
    elemento_primeira_linha_interno_para_shadow = f'[aria-label="palavra {variavel}"]'
    inner_shadow = shadow.find_element(By.CSS_SELECTOR, elemento_primeira_linha_interno_para_shadow).shadow_root


    for variavel in range(5):
        elemento_primeira_letra = f'[termo-pos="{str(variavel)}"]'
        elemento_html = inner_shadow.find_element(By.CSS_SELECTOR, elemento_primeira_letra)
        resposta = elemento_html.get_attribute('class')
        letra = remove_acentos(elemento_html.text.lower())
        if resposta != 'letter empty':
            lista_da_palavra.append([resposta, letra])


    return dict(zip(range(len(lista_da_palavra)), lista_da_palavra))

In [31]:
variavel = 1
elemento_do_board = '[id="board0"]'

shadow = driver.find_element(By.CSS_SELECTOR, elemento_do_board).shadow_root


lista_da_palavra = [] #inicializa a lista
elemento_primeira_linha_interno_para_shadow = f'[aria-label="palavra {variavel}"]'
inner_shadow = shadow.find_element(By.CSS_SELECTOR, elemento_primeira_linha_interno_para_shadow).shadow_root


for variavel in range(5):
    elemento_primeira_letra = f'[termo-pos="{str(variavel)}"]'
    elemento_html = inner_shadow.find_element(By.CSS_SELECTOR, elemento_primeira_letra)
    resposta = elemento_html.get_attribute('class')
    letra = elemento_html.text.lower()
    if resposta != 'letter empty':
        lista_da_palavra.append([resposta, letra])


resultado_teste = dict(zip(range(len(lista_da_palavra)), lista_da_palavra))
resultado_teste

{0: ['letter wrong', 't'],
 1: ['letter wrong', 'ó'],
 2: ['letter wrong', 'r'],
 3: ['letter place', 'i'],
 4: ['letter right', 'o']}

In [89]:
def descompacta_dicionario(dicionario, letras_aceitas):

    letras_erradas = ''
    letras_lugar_errado = []
    letras_corretas = []

    for item in dicionario:
        letra = remove_acentos(dicionario[item][1]).lower()
        condicao = letra in letras_aceitas
        if dicionario[item][0] == 'letter wrong' and (condicao == False):
            letras_erradas += str(dicionario[item][1])
        elif dicionario[item][0] == 'letter place':
            letras_aceitas += letra
            letras_lugar_errado.append([dicionario[item][1], item])
        elif dicionario[item][0] == 'letter right':
            letras_aceitas += letra
            letras_corretas.append([dicionario[item][1], item])
            
    return letras_erradas, letras_lugar_errado, letras_corretas, letras_aceitas

In [74]:
letra = 'o'
letras_aceitas = 'onz'

letra in letras_aceitas

True

In [ ]:
def adiciona_letras_aceitas(letra):
    string += letra
    return string

In [ ]:
def retorna_letras_aceitas():
    return 

In [21]:
#função que vai dizer se ta funcionando ou não
def get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa):
#def get_notificacao(palavra_sorteada, df):
    vencedor = 'wc-notify'
    notificacao = driver.find_element(By.CSS_SELECTOR, vencedor)
    notificacao = notificacao.text
    #print(notificacao)
    terminou = True
    if notificacao == '':
        notificacao = 'palavra aceita'
        terminou = True
        
        
    elif notificacao == 'essa palavra não é aceita':
        #file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
        #nesse ponto precisei meio que criar ela recursivamente, pra ir filtrando as palavras que não poderiamos colocar!
        while terminou == True:
            file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
            #aqui vai ter uma mini função, que talvez eu encapsule em outro lugar
            df = df[df.palavra != palavra_sorteada]
            palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())

            x_path_board = '/html/body'
            board_inicial = driver.find_element('xpath',x_path_board)

            for i in range(5):
                board_inicial.send_keys(Keys.BACKSPACE)
                sleep(1)

            envia_palavra(palavra_sorteada)
            dicionario = retorna_dicionario_respostas(tentativa)
            letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

            df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
            df = letra_na_posicao(letras_corretas, df)
            df = letras_nao_aceitas(letras_erradas, df)
            
            terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id,dicionario,tentativa)
            
            
        
        
    else:
        terminou = False
        
    return terminou, df, notificacao, palavra_sorteada

FAZENDO UMA SEGUNDA VERSÃO DA FUNCAO ACIMA

In [96]:
#função que vai dizer se ta funcionando ou não
def get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas):
#def get_notificacao(palavra_sorteada, df):
    vencedor = 'wc-notify'
    notificacao = driver.find_element(By.CSS_SELECTOR, vencedor)
    notificacao = notificacao.text
    #print(notificacao)
    terminou = True
    if notificacao == '':
        notificacao = 'palavra aceita'
        terminou = True
        
        
    elif notificacao == 'essa palavra não é aceita':
        #file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
        #nesse ponto precisei meio que criar ela recursivamente, pra ir filtrando as palavras que não poderiamos colocar!
        while terminou == True:
            
            
            #apagando palavra
            apaga_palavra()
            
            
            file.write(f'{-1},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
            #aqui vai ter uma mini função, que talvez eu encapsule em outro lugar
            df = df[df.palavra != palavra_sorteada].reset_index(drop=True)
            palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
            
            #palavra_sorteada = sorteia_palavra(df.palavra)

            envia_palavra(palavra_sorteada)

            #pegar informações da palavra enviada
            dicionario = retorna_dicionario_respostas(tentativa)
            print('-------- dentro do get_noficacao ------')
            print('palavra_sorteada: ',palavra_sorteada)
            print()
            print('dicionario: ', dicionario)
            print()
            print('letras_aceitas: ',letras_aceitas)
            print('tentativa: ', tentativa)
            print('-------- fim do get_noficacao ------')
            #df.to_csv('debugando.csv')
            df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
            #df.to_csv('debugando.csv')
            
            #terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id,dicionario,tentativa, letras_aceitas)
            terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
            tentativa += 1 # foi o jeito de corrigir o erro, não sei como só isso resolveu, verificar deep dps
            
            
        
        
    else:
        terminou = False
        
    return terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa

In [13]:
def first_attempt(df):
    melhores_palavras_para_vogais = df[df.vogais_unicas == 4]
    return sorteia_palavra(melhores_palavras_para_vogais.palavra.values.tolist())

In [87]:
def filtra_df(dicionario, df, letras_aceitas):
#         envia_palavra(palavra_sorteada)

#         #pegar informações da palavra enviada
#         dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas, letras_aceitas = descompacta_dicionario(dicionario, letras_aceitas)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        
        return df, letras_aceitas

# teste de inicialização e resolução

In [152]:
caminho = 'br-utf8.csv'
df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

while terminou == True:
    driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

    terminou, df, notificacao = get_notificacao(palavra_sorteada, df)
    print(f'tentativa: {tentativa} - palavra: {palavra_sorteada} - resultado: {notificacao}')
    tentativa += 1

ureia
tentativa: 0 - palavra: ureia - resultado: 
tentativa: 1 - palavra: zonzo - resultado: 
tentativa: 2 - palavra: tolho - resultado: 
tentativa: 3 - palavra: pombo - resultado: Fenomenal


In [155]:
caminho = 'br-utf8.csv'
df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

    terminou, df, notificacao = get_notificacao(palavra_sorteada, df)
    print(f'tentativa: {tentativa} - palavra: {palavra_sorteada} - resultado: {notificacao}')
    tentativa += 1

tentativa: 0 - palavra: vaiou - resultado: palavra aceita
tentativa: 1 - palavra: bolos - resultado: palavra aceita
tentativa: 2 - palavra: rombo - resultado: palavra aceita
tentativa: 3 - palavra: pombo - resultado: Fenomenal


In [14]:
from datetime import datetime

file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

            
    terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id}\n')
    tentativa += 1

file.close()

TypeError: get_notificacao() missing 1 required positional argument: 'dicionario'

# BLOCO DE CODIGO FINAL QUE EXECUTA E ESCREVE O QUE FOI FEITO

In [16]:
file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = random.randint(0,1000)
a = random.seed(variavel_randomica)
print(variavel_randomica)


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = descompacta_dicionario(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

            
    terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
    tentativa += 1

file.close()

416


TypeError: get_notificacao() missing 1 required positional argument: 'dicionario'

In [ ]:
dicionario_teste = {0: ['letter wrong', 'm'], 1: ['letter wrong', 'o'], 2: ['letter wrong', 'g'], 3: ['letter place', 'n'], 4: ['letter right', 'o']}

df = filtra_df(dicionario_teste, df)

In [98]:
letras_aceitas = ''

file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = 416
a = random.seed(variavel_randomica)
print(variavel_randomica)


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)
        
        #print(palavra_sorteada)
        df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
        
        #driver.implicitly_wait(10)
        sleep(1)
        terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)
        
        #print(palavra_sorteada)
        df, letras_aceitas = filtra_df(dicionario, df, letras_aceitas)
        #driver.implicitly_wait(30)
        sleep(1)
        terminou, df, notificacao, palavra_sorteada, letras_aceitas, tentativa = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    print('tentativa: ',tentativa)
    print('palavra sorteada: ',palavra_sorteada) 
    print('letras aceitas: ',letras_aceitas)
    #terminou, df, notificacao, palavra_sorteada, letras_aceitas = get_notificacao(palavra_sorteada, df, file, now, session_id, dicionario,tentativa, letras_aceitas)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica},{dicionario}\n')
    tentativa += 1

file.close()

416
tentativa:  0
palavra sorteada:  apeou
letras aceitas:  o
-------- dentro do get_noficacao ------
palavra_sorteada:  torio

dicionario:  {0: ['letter wrong', 't'], 1: ['letter wrong', 'o'], 2: ['letter wrong', 'r'], 3: ['letter place', 'i'], 4: ['letter right', 'o']}

letras_aceitas:  o
tentativa:  1
-------- fim do get_noficacao ------
-------- dentro do get_noficacao ------
palavra_sorteada:  linho

dicionario:  {0: ['letter wrong', 'l'], 1: ['letter right', 'i'], 2: ['letter right', 'n'], 3: ['letter wrong', 'h'], 4: ['letter right', 'o']}

letras_aceitas:  oio
tentativa:  2
-------- fim do get_noficacao ------
-------- dentro do get_noficacao ------
palavra_sorteada:  vindo

dicionario:  {0: ['letter wrong', 'v'], 1: ['letter right', 'i'], 2: ['letter right', 'n'], 3: ['letter wrong', 'd'], 4: ['letter right', 'o']}

letras_aceitas:  oioino
tentativa:  3
-------- fim do get_noficacao ------
-------- dentro do get_noficacao ------
palavra_sorteada:  finjo

dicionario:  {0: ['let

In [63]:
df_debug = pd.read_csv('debugando.csv')
df_debug

,Unnamed: 0,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4
0,0,chico,2,2,c,h,i,c,o
1,1,filho,2,2,f,i,l,h,o
2,2,golfo,2,1,g,o,l,f,o
3,3,kioto,3,2,k,i,o,t,o
4,4,bicho,2,2,b,i,c,h,o
...,...,...,...,...,...,...,...,...,...
282,282,obito,3,2,o,b,i,t,o
283,283,obolo,3,1,o,b,o,l,o
284,284,odios,3,2,o,d,i,o,s
285,285,otico,3,2,o,t,i,c,o


In [65]:
df_debug = pd.read_csv('debugando.csv')
dicionario_teste = {0: ['letter wrong', 'm'], 1: ['letter wrong', 'o'], 2: ['letter wrong', 'g'], 3: ['letter place', 'n'], 4: ['letter right', 'o']}


letras_erradas, letras_lugar_errado, letras_corretas = descompacta_dicionario(dicionario_teste)
print(letras_erradas)
print(letras_lugar_errado)
print(letras_corretas)

#achei o erro, algumas vezes ele adiciona a letra ja encontrada como letra errada
#solução: criar uma variavel que sempre guarda as letras corretas 
#e sempre passar elas letras corretas pra não adicionar como letter wrong!

mog
[['n', 3]]
[['o', 4]]


In [ ]:
df_debug = palavras_com_letra_posicao_errada(letras_lugar_errado, df_debug)
df_debug = letra_na_posicao(letras_corretas, df_debug)
df_debug = letras_nao_aceitas(letras_erradas, df_debug)
        
        
df_debug = filtra_df(dicionario_teste, df_debug)
df_debug

In [85]:
df_debug = pd.read_csv('debugando.csv')
dicionario_teste = {0: ['letter wrong', 'm'], 1: ['letter wrong', 'o'], 2: ['letter wrong', 'g'], 3: ['letter place', 'n'], 4: ['letter right', 'o']}


letras_erradas, letras_lugar_errado, letras_corretas = descompacta_dicionario(dicionario_teste, letras_aceitas)
print(letras_erradas)
print(letras_lugar_errado)
print(letras_corretas)

mog
[['n', 3]]
[['o', 4]]


In [209]:
file = open('resultados.txt', 'a')

caminho = 'br-utf8.csv'

df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()


terminou = True
tentativa = 0

now = datetime.now()
data = now.strftime("%d/%m/%Y")
#hora = now.strftime("%H:%M")
session_id = driver.session_id

#só verificando o seed que foi gerado pra caso precisar replicar
variavel_randomica = 52 #testando o q deu errado!
a = random.seed(variavel_randomica)
print(variavel_randomica)


while terminou == True:
    #driver.implicitly_wait(5)
    #for tentativa in range(5):
    #print(tentativa)
    if tentativa == 0:
        #print('first')
        #pegando a primeira palavra
        palavra_sorteada = first_attempt(df)
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(10)
        sleep(2)
    else:

        palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
        #print(palavra_sorteada)

        #enviando palavra para o site
        envia_palavra(palavra_sorteada)

        #pegar informações da palavra enviada
        dicionario = retorna_dicionario_respostas(tentativa)

        #retornar resultado das letras
        letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

        df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
        df = letra_na_posicao(letras_corretas, df)
        df = letras_nao_aceitas(letras_erradas, df)
        #driver.implicitly_wait(30)
        sleep(2)

            
    terminou, df, notificacao, palavra_sorteada = get_notificacao(palavra_sorteada, df, file, now, session_id)
    
    #file.write('tentativa,palavra,resultado,data,session_id\n')
    file.write(f'{tentativa},{palavra_sorteada},{notificacao},{data},{session_id},{variavel_randomica}\n')
    tentativa += 1

file.close()

52


In [200]:

variavel_randomica = random.randint(0,1000)
print(variavel_randomica)
a = random.seed(variavel_randomica)
random.randint(0,1000)

664


804

In [199]:
a = random.seed(821)
random.randint(0,1000)

583

In [201]:
a = random.seed(664)
random.randint(0,1000)

804

In [150]:
#depois procurar pq nao ta funcionando essa funçaõ

caminho = 'br-utf8.csv'
df = carrega_dataframe_csv(caminho)

driver = inicializa_termo()

def resolve():
    terminou = True
    tentativa = 0

    while terminou == True:
        print(tentativa)
        if tentativa == 0:
            print('first')
            #pegando a primeira palavra
            palavra_sorteada = first_attempt(df)
            print(palavra_sorteada)

            #enviando palavra para o site
            envia_palavra(palavra_sorteada)

            #pegar informações da palavra enviada
            dicionario = retorna_dicionario_respostas(tentativa)

            #retornar resultado das letras
            letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

            df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
            df = letra_na_posicao(letras_corretas, df)
            df = letras_nao_aceitas(letras_erradas, df)
            #driver.implicitly_wait(10)
            sleep(2)
        else:

            palavra_sorteada = sorteia_palavra(df.palavra.values.tolist())
            print(palavra_sorteada)

            #enviando palavra para o site
            envia_palavra(palavra_sorteada)

            #pegar informações da palavra enviada
            dicionario = retorna_dicionario_respostas(tentativa)

            #retornar resultado das letras
            letras_erradas, letras_lugar_errado, letras_corretas = retorna_resultado_tentativa_palavra(dicionario)

            df = palavras_com_letra_posicao_errada(letras_lugar_errado, df)
            df = letra_na_posicao(letras_corretas, df)
            df = letras_nao_aceitas(letras_erradas, df)
            #driver.implicitly_wait(30)
            sleep(2)

        terminou, df, notificacao = get_notificacao(palavra_sorteada, df)
        print(notificacao)
        tentativa += 1

# proximo passo seria pegar todas as palavras que mandou pro site e guardar tbm, pra tentar gerar um machine learning